# Spring2025_s3fs.example.ipynb

This notebook provides an example use case integrating all core functionality of the Bitcoin time series project using `s3fs`. It demonstrates data ingestion, S3 operations, visualization, and basic forecasting.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import s3fs
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
fs = s3fs.S3FileSystem(anon=False)
s3_path = 'bitcoin-timeseries-data-kv/bitcoin_prices.csv'
with fs.open(s3_path, mode='rb') as f:
    df = pd.read_csv(f)

print("✅ Data loaded from S3")
df.head()

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Price'], label='Bitcoin Price')
plt.title('Bitcoin Price Time Series')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
result = seasonal_decompose(df['Price'], model='additive', period=7)
result.plot()
plt.suptitle('Bitcoin Price Decomposition (Trend + Seasonality + Residuals)', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
df['Moving_Avg'] = df['Price'].rolling(window=5).mean()
with fs.open('bitcoin-timeseries-data-kv/bitcoin_prices_with_forecast.csv', 'w') as f:
    df.to_csv(f)
print("✅ Forecasted data uploaded to S3")

### ✅ This notebook demonstrates an end-to-end flow combining the API, file read/write with `s3fs`, time series processing, and forecasting.